# 02. 문자 단위 RNN(Char RNN) - 더 많은 데이터
---

### 2. 문자 단위 RNN(Char RNN)

In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [37]:
# 1. 훈련 데이터 전처리하기

sentence = "if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."

In [38]:
# 문자 집합 생성하고 각 문자에 정수 부여 
char_set = list(set(sentence))   # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)}   # enumerate로 각 문자에 정수 인코딩

In [39]:
print(char_dic)
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))
# 3번 공백과 21번 콤마 등도 하나의 원소로 들어감

{'b': 0, 'n': 1, 'm': 2, ' ': 3, 'o': 4, '.': 5, 'y': 6, 'g': 7, 'f': 8, 'l': 9, "'": 10, 'e': 11, 'i': 12, 'u': 13, 'c': 14, 'a': 15, 's': 16, 'd': 17, 'k': 18, 't': 19, 'h': 20, ',': 21, 'p': 22, 'r': 23, 'w': 24}
문자 집합의 크기 : 25


In [40]:
# 하이퍼파라미터 설정
hidden_size = dic_size  # 이 수는 임의로 바꿔도 무방
sequence_length = 10  # 10개 단위로 끊어서 샘플을 만들 예정
learning_rate = 0.1

In [41]:
# sequence_length = 10 처럼 10을 단위로 샘플들을 잘라서 데이터를 만들자

# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1:i+sequence_length+1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str]) 
    y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [42]:
# 데이터 확인
print(x_data[0])    # = if you wan
print(y_data[0])    # = f you want

[12, 8, 3, 6, 4, 13, 3, 24, 15, 1]
[8, 3, 6, 4, 13, 3, 24, 15, 1, 19]


In [44]:
# 정수데이터를 이용해 원-핫 인코딩 수행
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [45]:
# 데이터 크기 확인
print(f'훈련 데이터 크기: {X.shape}')
print(f'레이블 크기: {Y.shape}')

훈련 데이터 크기: torch.Size([170, 10, 25])
레이블 크기: torch.Size([170, 10])


In [49]:
print(X[0])
print(len(X[0]))

print(Y[0])
print(len(Y[0]))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [57]:
# 2. 모델 구현하기

# 이번에는 은닉층을 두 개 쌓아보자
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        
        x = self.fc(x)
        return x

In [58]:
net = Net(dic_size, hidden_size, 2) # 히든 레이어가 두 개

In [59]:
# 비용 함수와 옵티마이저 선언
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [60]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

# torch.Size([170, 10, 25])
# 170 배치차원
# 10 시점(timesteps)
# 25 출력의 크기

torch.Size([170, 10, 25])


In [65]:
# veiw를 사용해 차원 맞추기

print(outputs.shape)
print(outputs.view(-1, dic_size).shape,'\n') # 2차원 텐서로 변환.
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10, 25])
torch.Size([1700, 25]) 

torch.Size([170, 10])
torch.Size([1700])


In [66]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

....b.b.....ebb.....ebbb..........b..b....b..w...b.b....b.b.bb.b..e..b.bb..b........b..e......b....bbe.....ebeb....eb.beb..b....eb..b...bb.be..e..b..ew....eb...........b.bw..ew...
 h hdhdhddhdhhdhhhdhdhdhdhhhdhddhhdhhdhdhhdhdhdhhdhhhdhhhhdhddhhhhhhhdhdhhdhdhdhddhhdhhhdhdhhhhhdhhhhhdhdhdhdhhhdhdhdhdhhdhdhhhdhdhhhhdhddhhdhdhdhdhhhdhhhddhhdhhhdhhdhhdhdhhhhdhhh
                                       t                                                                        t                                                                  
tototttottottttttttotototttototototottotottottttttotttottotottttttttotottotototototottttttttottototttttototototototototottototttotottototttttototototttttotttttotttttttottttotttttt
  roh dmd h od   h  d d d d d h h d h h o  h h h   d d h htod d h h   od    d d h h d h d h     d   h d d d d h d h d d h h  d d d d d hd d h d d d d d h h   h   h       h      h 
      r r    i d  i d d d d d i i   d  d  i   d  d   i    d r d   d  r r  d   d i i   d i d   i r i 